In [3]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [4]:
len(documents)

1103663

In [5]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [6]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [13]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(42)

In [15]:
test = WordNetLemmatizer()
test.lemmatize('went', pos='v')

'go'

In [18]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(word) for word in original_words]
pd.DataFrame({'Orig_words': original_words, 'Single': singles})

,Orig_words,Single
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [44]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    
    return result

In [45]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('Original Documents')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print(preprocess(doc_sample))

Original Documents
['rain', 'helps', 'dampen', 'bushfires']
['rain', 'help', 'dampen', 'bushfir']


In [46]:
processed_docs = documents['headline_text'].map(preprocess)

In [47]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

#### This will give the BOW

In [48]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [51]:
count = 0
for k, v in dictionary.iteritems():
    print(k,v)
    count +=1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [54]:
print(dictionary)

Dictionary(62240 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)


In [55]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [56]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [57]:
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [64]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


In [68]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [71]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908867507543),
 (1, 0.38929654337861147),
 (2, 0.4964985175717023),
 (3, 0.5046520327464028)]


In [72]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [73]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"govern" + 0.018*"live" + 0.016*"council" + 0.014*"water" + 0.013*"deal" + 0.012*"busi" + 0.012*"plan" + 0.011*"budget" + 0.011*"want" + 0.011*"royal"
Topic: 1 
Words: 0.020*"help" + 0.018*"hospit" + 0.014*"work" + 0.013*"worker" + 0.012*"power" + 0.011*"urg" + 0.011*"farm" + 0.010*"health" + 0.010*"fund" + 0.008*"union"
Topic: 2 
Words: 0.023*"adelaid" + 0.020*"perth" + 0.020*"open" + 0.017*"final" + 0.015*"tasmanian" + 0.010*"centr" + 0.010*"peopl" + 0.009*"announc" + 0.009*"research" + 0.009*"drum"
Topic: 3 
Words: 0.022*"market" + 0.019*"south" + 0.016*"west" + 0.015*"share" + 0.013*"train" + 0.013*"bank" + 0.011*"victorian" + 0.010*"build" + 0.010*"life" + 0.010*"take"
Topic: 4 
Words: 0.033*"charg" + 0.031*"court" + 0.024*"murder" + 0.020*"face" + 0.018*"alleg" + 0.017*"jail" + 0.016*"accus" + 0.016*"woman" + 0.014*"child" + 0.014*"death"
Topic: 5 
Words: 0.017*"countri" + 0.016*"nation" + 0.016*"rural" + 0.014*"hour" + 0.013*"servic" + 0.013*"minist" + 0.0

In [74]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [75]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}'.format(idx, topic))

Topic: 0 
Word: 0.012*"market" + 0.008*"share" + 0.006*"australian" + 0.006*"climat" + 0.005*"rise" + 0.005*"tasmania" + 0.005*"ash" + 0.005*"murray" + 0.005*"price" + 0.005*"decemb"
Topic: 1 
Word: 0.018*"charg" + 0.016*"polic" + 0.016*"murder" + 0.015*"trump" + 0.012*"court" + 0.012*"woman" + 0.011*"alleg" + 0.010*"jail" + 0.010*"death" + 0.009*"assault"
Topic: 2 
Word: 0.010*"final" + 0.009*"world" + 0.008*"leagu" + 0.006*"australia" + 0.006*"open" + 0.006*"win" + 0.005*"juli" + 0.005*"beat" + 0.005*"rugbi" + 0.005*"star"
Topic: 3 
Word: 0.014*"live" + 0.011*"drum" + 0.010*"weather" + 0.007*"violenc" + 0.007*"east" + 0.007*"septemb" + 0.007*"monday" + 0.006*"malcolm" + 0.006*"domest" + 0.006*"australia"
Topic: 4 
Word: 0.023*"countri" + 0.021*"hour" + 0.014*"rural" + 0.010*"turnbul" + 0.007*"farm" + 0.006*"wednesday" + 0.006*"farmer" + 0.006*"dairi" + 0.006*"nation" + 0.005*"bushfir"
Topic: 5 
Word: 0.017*"interview" + 0.010*"royal" + 0.009*"grandstand" + 0.007*"marriag" + 0.007*"fr

In [76]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [77]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6199999451637268	 
Topic: 0.020*"help" + 0.018*"hospit" + 0.014*"work" + 0.013*"worker" + 0.012*"power" + 0.011*"urg" + 0.011*"farm" + 0.010*"health" + 0.010*"fund" + 0.008*"union"

Score: 0.2200000286102295	 
Topic: 0.022*"market" + 0.019*"south" + 0.016*"west" + 0.015*"share" + 0.013*"train" + 0.013*"bank" + 0.011*"victorian" + 0.010*"build" + 0.010*"life" + 0.010*"take"

Score: 0.019999999552965164	 
Topic: 0.025*"govern" + 0.018*"live" + 0.016*"council" + 0.014*"water" + 0.013*"deal" + 0.012*"busi" + 0.012*"plan" + 0.011*"budget" + 0.011*"want" + 0.011*"royal"

Score: 0.019999999552965164	 
Topic: 0.023*"adelaid" + 0.020*"perth" + 0.020*"open" + 0.017*"final" + 0.015*"tasmanian" + 0.010*"centr" + 0.010*"peopl" + 0.009*"announc" + 0.009*"research" + 0.009*"drum"

Score: 0.019999999552965164	 
Topic: 0.033*"charg" + 0.031*"court" + 0.024*"murder" + 0.020*"face" + 0.018*"alleg" + 0.017*"jail" + 0.016*"accus" + 0.016*"woman" + 0.014*"child" + 0.014*"death"

Score: 0.019999999

In [78]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5066022872924805	 
Topic: 0.023*"countri" + 0.021*"hour" + 0.014*"rural" + 0.010*"turnbul" + 0.007*"farm" + 0.006*"wednesday" + 0.006*"farmer" + 0.006*"dairi" + 0.006*"nation" + 0.005*"bushfir"

Score: 0.3333844542503357	 
Topic: 0.012*"market" + 0.008*"share" + 0.006*"australian" + 0.006*"climat" + 0.005*"rise" + 0.005*"tasmania" + 0.005*"ash" + 0.005*"murray" + 0.005*"price" + 0.005*"decemb"

Score: 0.020003976300358772	 
Topic: 0.014*"live" + 0.011*"drum" + 0.010*"weather" + 0.007*"violenc" + 0.007*"east" + 0.007*"septemb" + 0.007*"monday" + 0.006*"malcolm" + 0.006*"domest" + 0.006*"australia"

Score: 0.020002350211143494	 
Topic: 0.008*"octob" + 0.008*"mother" + 0.006*"quiz" + 0.005*"drink" + 0.005*"light" + 0.005*"onlin" + 0.005*"grain" + 0.005*"driver" + 0.005*"port" + 0.005*"safe"

Score: 0.020001620054244995	 
Topic: 0.013*"govern" + 0.008*"health" + 0.008*"podcast" + 0.007*"elect" + 0.006*"sport" + 0.006*"fund" + 0.006*"say" + 0.006*"labor" + 0.005*"minist" + 0.005*"

In [79]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3500001132488251	 Topic: 0.025*"govern" + 0.018*"live" + 0.016*"council" + 0.014*"water" + 0.013*"deal"
Score: 0.1833333522081375	 Topic: 0.022*"market" + 0.019*"south" + 0.016*"west" + 0.015*"share" + 0.013*"train"
Score: 0.18333317339420319	 Topic: 0.051*"polic" + 0.024*"attack" + 0.021*"sydney" + 0.021*"kill" + 0.015*"crash"
Score: 0.18332935869693756	 Topic: 0.023*"adelaid" + 0.020*"perth" + 0.020*"open" + 0.017*"final" + 0.015*"tasmanian"
Score: 0.016670677810907364	 Topic: 0.017*"countri" + 0.016*"nation" + 0.016*"rural" + 0.014*"hour" + 0.013*"servic"
Score: 0.01666666753590107	 Topic: 0.020*"help" + 0.018*"hospit" + 0.014*"work" + 0.013*"worker" + 0.012*"power"
Score: 0.01666666753590107	 Topic: 0.033*"charg" + 0.031*"court" + 0.024*"murder" + 0.020*"face" + 0.018*"alleg"
Score: 0.01666666753590107	 Topic: 0.043*"australian" + 0.034*"trump" + 0.019*"school" + 0.019*"coast" + 0.017*"die"
Score: 0.01666666753590107	 Topic: 0.022*"elect" + 0.020*"canberra" + 0.017*"north"

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

